# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-predictions/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-predictions/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_e

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.",1,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.",1,finance,gpt-4o,NAVI_GATOR,0,2
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1
3,"On August 21, 2023, Goldman Sachs speculates that the stock price at Amazon will likely increase.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,2
4,"George, a financial analyst, predicts that on 01/15/2024, the Google revenue may rise.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,3
5,"According to BlackRock, the operating cash flow at ExxonMobil would fall in Q3 2023.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,4
6,"In Q2 2023, Wells Fargo envisions that the stock price at Tesla has some probability to remain stable.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,5


## Observations

In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observations
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observations/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observations
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observations/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictio

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"The financial reporter, Sarah, witnessed the stock price at Tesla plummeted in Q2 of 2023.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"From May 15, 2025, the financial analyst speculated the net profit at Amazon was drastically different.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,2
2,"On July 1, 2024, the financial expert heard that the revenue at Google remained stable.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,5
3,"Apple's operating income generally rose in August 21, 2027, according to the financial top executive.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,6
4,Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0,finance,gpt-4o,NAVI_GATOR,0,1
5,"From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0,finance,gpt-4o,NAVI_GATOR,0,2
6,"2026/12/01, a financial analyst heard that the operating income at Amazon remained stable.",0,finance,gpt-4o,NAVI_GATOR,0,5


## Both

- Create a knowledge graph
    - Nodes: words
    - Edges: connection to other words (same/diff sentence)
- Look at code from Graphbreeding project on 2019 Mac

In [5]:
df = DataProcessing.concat_dfs([predictions_df, observations_df])
df.head(3)

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.",1,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.",1,finance,gpt-4o,NAVI_GATOR,0,2
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1


In [6]:
predictions = DataProcessing.df_to_list(predictions_df, "Base Sentence")
observations = DataProcessing.df_to_list(observations_df, "Base Sentence")

In [7]:
# from text_generation_models import LlamaVersatileTextGenerationModel
# llama_versatile_generation_model = LlamaVersatileTextGenerationModel()

# from text_generation_models import TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# # llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# # llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')


# prompt = f"Can you return the observations ({observations}) that certify this prediction ({predictions[0]})? Only write the observations that certify the prediction."
# # prompt = f"Can you return a list of the observations ({observations}) that certify this prediction ({predictions[0]}) and why? Only write the observations that certify the prediction and why. Do not write any other text. "
# input_prompt = llama_8b_8192_generation_model.user(prompt)
# # print(input_prompt)
# # raw_text = self.chat_completion([self.user(prompt_template)])
# raw_text = llama_8b_8192_generation_model.chat_completion([input_prompt])
# print(raw_text)

In [8]:
# raw_text

In [9]:
# # Extract observations from the string
# observations = raw_text.split('\n\n')[1].split('\n')[1:-1]
# # Remove numbering and quotes
# observations = [obs.split('. ', 1)[1].strip('"') for obs in observations]
# # Print the list
# print(observations)


In [10]:
# type(observations)

In [11]:
# observations[1]

In [12]:
# from text_generation_models import TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# models = [llama_70b_8192_generation_model]
# # models = [llama_versatile_generation_model, llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]
# # Prompt for the model

# prompt = f"Can you return the observations ({observations}) that certify this prediction ({predictions[0]})? Write in the format of ({predictions[0]}, {observations})"
# input_prompt = llama_versatile_generation_model.user(prompt)
# # print(input_prompt)

# model_certify = {}
# for model in models:    
    
#     raw_text = model.chat_completion([input_prompt])
#     print(raw_text)

In [13]:
# df

In [14]:
# data = []
# for model, output in model_certify.items():
#     for o in output:
#         data.append([model, o])

# # Create the DataFrame
# df = pd.DataFrame(data, columns=['Model', 'Output'])
# df

In [15]:
# import pandas as pd
# from text_generation_models import TextGenerationModelFactory

# # Initialize the TextGenerationModelFactory
# tgmf = TextGenerationModelFactory()

# # Create instances of the models
# llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
# llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
# llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
# llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

# # List of models
# models = [llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]

# # Prompt for the model
# prompt = f"Can you return a list of the observations ({observations}) that certify this prediction ({predictions[0]})? Only write the observations that certify the prediction. Do not write any other text."
# input_prompt = llama_versatile_generation_model.user(prompt)

# # Dictionary to store model outputs
# model_certify = {}
# for model in models:
#     raw_text = model.chat_completion([input_prompt])
#     output = [line.strip().replace("*", "") for line in raw_text.split("\n") if line.strip()]
#     model_certify[model.model_name] = output

# # Prepare data for DataFrame
# data = []
# for model, output in model_certify.items():
#     for o in output:
#         if isinstance(o, list):
#             o = ', '.join(o)
#         data.append([predictions[0], model, o])

# # Create the DataFrame
# df = pd.DataFrame(data, columns=['Prediction', 'Model', 'Observations'])

# # Display the DataFrame
# df


In [16]:
# df

In [17]:
from text_generation_models import TextGenerationModelFactory
tgmf = TextGenerationModelFactory()

llama_versatile_generation_model = tgmf.create_instance(model_name='llama-3.3-70b-versatile')
llama_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant')
llama_70b_8192_generation_model = tgmf.create_instance('llama3-70b-8192')
llama_8b_8192_generation_model = tgmf.create_instance('llama3-8b-8192')

models = [llama_instant_generation_model, llama_70b_8192_generation_model, llama_8b_8192_generation_model]
# Prompt for the model

prompt = f"Return a list of the observations ({observations}) that certify this prediction ({predictions[0]})?"
input_prompt = llama_versatile_generation_model.user(prompt)
# print(input_prompt)

# df = pd.DataFrame(columns=["Model", "Prompt", "Response"])
model_certify = {}
for model in models:    
    
    raw_text = model.chat_completion([input_prompt])
    output = []
    for line in raw_text.split("\n"):
        if line.strip():  # Skip empty lines
            output.append(line.strip())
    # print(output)
    model_certify[model.model_name] = output

print(model_certify)
model_certify.keys()

data = []
for model, output in model_certify.items():
    for output in output:
        data.append([model, output])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Model', 'Output'])
df

{'llama-3.1-8b-instant': ['To verify the prediction, we need to analyze the given observations and find any that match the prediction. Here are the observations that certify this prediction:', "1. 'On August 21, 2024, a college student heard that the revenue of Amazon remained stable.'", "2. 'From Q2 2024, Goldman Sachs' research department speculated that the stock price at Amazon was drastically different.'", "3. 'On August 21, 2024, a college student heard that the revenue of Amazon remained stable.'", "4. 'The financial reporter, Sarah, witnessed the stock price at Amazon plummeted in Q2 of 2023.'", "5. 'From May 15, 2025, the financial analyst speculated the net profit at Amazon was drastically different.'", "6. 'On July 1, 2024, the financial expert heard that the revenue at Google remained stable.'", "7. 'Amazon's operating income generally rose in August 21, 2024, according to the financial top executive.'", "8. 'From 2025-03-30, Morgan Stanley speculated the gross profit at Ap

,Model,Output
0,llama-3.1-8b-instant,"To verify the prediction, we need to analyze the given observations and find any that match the prediction. Here are the observations that certify this prediction:"
1,llama-3.1-8b-instant,"1. 'On August 21, 2024, a college student heard that the revenue of Amazon remained stable.'"
2,llama-3.1-8b-instant,"2. 'From Q2 2024, Goldman Sachs' research department speculated that the stock price at Amazon was drastically different.'"
3,llama-3.1-8b-instant,"3. 'On August 21, 2024, a college student heard that the revenue of Amazon remained stable.'"
4,llama-3.1-8b-instant,"4. 'The financial reporter, Sarah, witnessed the stock price at Amazon plummeted in Q2 of 2023.'"
5,llama-3.1-8b-instant,"5. 'From May 15, 2025, the financial analyst speculated the net profit at Amazon was drastically different.'"
6,llama-3.1-8b-instant,"6. 'On July 1, 2024, the financial expert heard that the revenue at Google remained stable.'"
7,llama-3.1-8b-instant,"7. 'Amazon's operating income generally rose in August 21, 2024, according to the financial top executive.'"
8,llama-3.1-8b-instant,"8. 'From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.'"
9,llama-3.1-8b-instant,"9. '2026/12/01, a financial analyst heard that the operating income at Amazon remained stable.'"
